## Table of Contents

- [Size Functions](#Size-Functions)
  - [size_function_from_depth()](#size_function_from_depth())
  - [size_function_from_edge_lengths()](#size_function_from_edge_lengths())
- [Smooth Size Functions](#Smooth-Size-Functions)
  - [smooth_size_function()](#smooth_size_function())
  - [smooth_size_function_ugrid()](#smooth_size_function_ugrid())
- [Smooth Elevation Functions](#Smooth-Elevation-Functions)
  - [smooth_elev_by_slope()](#smooth_elev_by_slope())
  - [smooth_elev_by_slope_ugrid()](#smooth_elev_by_slope_ugrid())
- [generate_mesh()](#generate_mesh())
- [generate_2dm()](#generate_2dm())
- [check_mesh_input_topology()](#check_mesh_input_topology())
- [redistribute_poly_line()](#redistribute_poly_line())

In [1]:
from xms.mesher.meshing import mesh_utils
from xms.mesher.meshing import poly_input
from xms.mesher.meshing.multi_poly_mesher_io import MultiPolyMesherIo

## Size Functions

A size function is a multiple that guides the size of elements to be created in SMS. A size function determines the element size based off of a dataset that will be created by SMS. Each point is assigned a size value. This size value is the approximate size of the elements to be created in the region where the point is located. The mesh will be denser where the size values are smaller. The size function dataset can then be used to redistribute vertices along an arc or used as the bathymetry for polygons. Size functions can be based off of different criteria. For example, they may be based on either depth, slope, or curvature of the model. For more information on size functions see the <a href="https://www.xmswiki.com/wiki/SMS:Size_Function">xmswiki</a>.

## size_function_from_depth()

The `size_function_from_depth()` function creates a size for each given depth based on the depths passed. Each depth corresponds to a point on the model, and each size function returned corresponds to the same point for the corresponding depth. This function is often useful for coastal numerical model simulations.  `size_function_from_depth()` takes the following arguments:

- `depths` (iterable): The measured depths at point locations.
- `min_size` (float): The minimum element edge size.
- `max_size` (float): The maximum element edge size.

`size_function_from_depth()` returns a tuple of sizes, one for each depth passed to the `depths` argument.

In [2]:
depths = (0, 5, 10, 20, 25, 5, 0)
min_elem = 2
max_elem = 102

mesh_utils.size_function_from_depth(depths, min_elem, max_elem)

(2.0, 22.0, 42.0, 82.0, 102.0, 22.0, 2.0)

## size_function_from_edge_lengths()

The `size_function_from_edge_lengths()` creates a size for each point in the given grid based on the average length of the connected edges to the point. `size_function_from_edge_lengths()` takes the following argument:

- `ugrid` (UGrid): The unstructured grid.

`size_function_from_edge_lengths()` returns a tuple of sizes whose length is equal to the number of points in the given grid.

In [3]:
# Import needed to read grid in from file
from xms.grid.ugrid import ugrid_utils

# Read in grid
in_ugrid = ugrid_utils.read_ugrid_from_ascii_file('grids/ugrid_1.xmugrid')

# Get sizes
sizes = mesh_utils.size_function_from_edge_lengths(in_ugrid)

# Print selection of sizes
for size in sizes[:5]:
    print(size)
    
print('...')

10.83564369593963
10.23665819865666
10.114041996296901
9.954311679197799
9.506993258291434
...


## Smooth Size Functions

Smoothing a dataset is used to condition scattered data scalar values before those values are used in an interpolation process. One measure of mesh quality is element area change. If the dataset values change too quickly in a size dataset, the element area change of adjacent elements may be too great, resulting in poor mesh quality. We can smooth a size dataset to prevent the dataset values from changing too quickly. For more information see the <a href="https://www.xmswiki.com/wiki/SMS:Smooth_Dataset">xmswiki</a>. xmsmesher provides utility functions that smooth size datasets. These functions are shown below.

## smooth_size_function()

The `smooth_size_function()` function ensures that the size function transitions over a sufficient distance so that the area change of adjacent elements meets the size ratio passed in. The `smooth_size_function()` takes the following arguments:

- `tin` (tin): Points and triangles defining the connectivity of the size function.
- `sizes` (iterable): Array of the current sizes.
- `size_ratio` (float): Allowable size difference between adjacent elements.
- `min_size` (float): Minimum specified element size.
- `anchor_to` (str): Option to anchor to the minimum or maximum size (`'min'` or `'max'`).
- `points_flag` (iterable): Flag to indicate if the value at the point should be adjusted (a value of true will skip the point). Leave the bitset empty to process all points.

An array of smoothed sizes is returned. A simple example of how to use the `smooth_size_function()` is given below.

In [4]:
# Import needed for Tin
from xms.grid.triangulate import Tin

# Create a Tin
pts = ((0, 0, 0), (10, 0, 0), (20, 0, 0), (30, 0, 0), (0, 10, 0), (10, 10, 0),
       (20, 10, 0), (30, 10, 0), (0, 20, 0), (10, 20, 0), (20, 20, 0), (30, 20, 0))

tris = ()
adj_tris = ()

tin = Tin(pts, tris)
tin.triangles_adjacent_to_points = adj_tris
tin.triangulate()

# Create inputs for function
sizes = [1 for _ in range(0, 12)]
sizes[4] = 100

size_ratio = 0.5
min_size = 1.0
anchor_type = 'max'
pt_flags = ()

# Smooth the sizes
mesh_utils.smooth_size_function(tin, sizes, size_ratio, min_size, anchor_type, pt_flags)

(96.53426361083984,
 95.09870910644531,
 91.63297271728516,
 88.167236328125,
 100.0,
 96.53426361083984,
 93.06852722167969,
 89.60279083251953,
 96.53426361083984,
 93.06852722167969,
 89.60279083251953,
 86.13705444335938)

## smooth_size_function_ugrid()

The `smooth_size_function_ugrid()` Ensures that the size function transitions over a sufficient distance so that the area change of adjacent elements meets the size ratio passed in. It is the same as the `smooth_size_function()` function except it accepts a `UGrid` instead of a `Tin`. The `smooth_size_function_ugrid()` accepts the following arguments:

- `ugrid` (UGrid): Unstructured grid defining the connectivity of the size function.
- `sizes` (iterable): Array of the current sizes.
- `size_ratio` (float): Allowable size difference between adjacent elements.
- `min_size` (float): Minimum specified element size.
- `anchor_to` (str): Option to anchor to the minimum or maximum size ('min' or 'max')
- `points_flag` (iterable): Flag to indicate if the value at the point should be adjusted (a value of true will skip the point). Leave the bitset empty to process all points.

An array of smoothed sizes is returned. A simple example of how to use the `smooth_size_function_ugrid()` is given below.

In [5]:
# Create inputs for function
sizes = [1 for _ in range(0, 158)]
sizes[4] = 100

size_ratio = 0.5
min_size = 1.0
anchor_type = 'max'
pt_flags = ()

# Smooth the sizes (Note: in_ugrid is a UGrid we read in from a file earlier in the notebook)
smooth_sizes = mesh_utils.smooth_size_function_ugrid(in_ugrid, sizes, size_ratio, min_size, anchor_type, pt_flags)

# Print the first 10 sizes from the smoothed sizes returned
for size in smooth_sizes[:10]:
    print(size)
    
print('...')

86.13705444335938
89.60279083251953
93.06852722167969
96.53426361083984
100.0
96.53426361083984
93.06852722167969
89.60279083251953
86.13705444335938
82.67131805419922
...


## Smooth Elevation Functions

xmsmesher provides utility functions that smooth the elevation of a dataset. These can be used to smooth depth/elevation values to prevent extreme slopes. These functions are shown below.

## smooth_elev_by_slope()

The `smooth_elev_by_slope()` function smooths elevations based on a max specified slope. It takes the following arguments:

- `tin` (tin): Points and triangles defining the connectivity of the elevations.
- `elevations` (iterable): Array of the current elevations.
- `max_slope` (float): Maximum allowable slope.
- `anchor_to` (str): Option to anchor to the minimum or maximum size (`'min'` or `'max'`).
- `points_flag` (iterable): Flag to indicate if the value at the point should be adjusted (a value of true will skip the point). Leave the bitset empty to process all points.

An array of smoothed elevations is returned.  A simple example showing how to use `smooth_elev_by_slope()` is given below.

In [6]:
# Create a Tin
pts = ((0, 0, 0), (10, 0, 0), (20, 0, 0), (30, 0, 0), (0, 10, 0), (10, 10, 0),
       (20, 10, 0), (30, 10, 0), (0, 20, 0), (10, 20, 0), (20, 20, 0), (30, 20, 0))

tris = ()
adj_tris = ()

tin = Tin(pts, tris)
tin.triangles_adjacent_to_points = adj_tris
tin.triangulate()

# Create inputs for smooth_elev_by_slope()
elevations = [100 for _ in range(0, 12)]
elevations[4] = 1
min_size = 0.5
anchor_type = 'min'
pt_flags = ()

# Smooth the elevations
mesh_utils.smooth_elev_by_slope(tin, elevations, min_size, anchor_type, pt_flags)

(6.0,
 8.071067810058594,
 13.071067810058594,
 18.071067810058594,
 1.0,
 6.0,
 11.0,
 16.0,
 6.0,
 11.0,
 16.0,
 21.0)

## smooth_elev_by_slope_ugrid()

The `smooth_elev_by_slope_ugrid()` function smooths elevations based on a max specified slope. It is the same as `smooth_elev_by_slope()` except it takes a `UGrid` instead of a `Tin`. It takes the following arguments:

- `ugrid` (UGrid): Unstructured grid defining the connectivity of the elevations.
- `elevations` (iterable): Array of the current elevations.
- `max_slope` (float): Maximum allowable slope.
- `anchor_to` (str): Option to anchor to the minimum or maximum size ('min' or 'max')
- `points_flag` (iterable): Flag to indicate if the value at the point should be adjusted (a value of true will skip the point). Leave the bitset empty to process all points.

An array of smoothed elevations is returned.  A simple example showing how to use `smooth_elev_by_slope_ugrid()` is given below.

In [7]:
# Create inputs for function
elevs = [1 for _ in range(0, 158)]
elevs[4] = 100

min_slope = 1.0
anchor_type = 'max'
pt_flags = ()

# Smooth the elevations (Note: in_ugrid is a UGrid we read in from a file earlier in the notebook)
smooth_elevs = mesh_utils.smooth_elev_by_slope_ugrid(in_ugrid, elevs, min_slope, anchor_type, pt_flags)

# Print the first 10 elevations from the smoothed elevations returned
for elev in smooth_elevs[:10]:
    print(elev)

print('...')

60.0
70.0
80.0
90.0
100.0
90.0
80.0
70.0
60.0
50.0
...


## generate_mesh()

The `generate_mesh()` function creates a mesh from input polygons. It takes the following argument:

- `mesh_io` (MultiPolyMesherIo): Input polygons and options for generating a mesh.

See the other notebooks in xmsmesher (MeshSimplePolygon, MeshScalarPaving, and MeshSanDiego) for examples on how to use `generate_mesh()`.

## generate_2dm()

The `generate_2dm()` function allows you to create a mesh from input polygons and write it to a 2dm file. It takes the following arguments:

- `mesh_io` (MultiPolyMesherIo): Input polygons and options for generating a mesh.
- `file_name` (str): The file name of the output 2dm file.
- `precision` (int, optional): The decimal point precision of the resulting mesh.

The function returns a tuple where the first entry is `True` if the mesh was generated successfully, and `False` otherwise. The second entry in the returned tuple is a string of messages in the case where the mesh wasn't generated successfully.

In [8]:
polygon_points = [
    (0, 10,0),   (0, 20,0),   (0, 30,0),   (0, 40,0),    (0, 50,0),   (0, 60,0),   (0, 70,0),   (0, 80,0),
    (0, 90,0),   (0, 100,0),  (10, 100,0), (20, 100,0),  (30, 100,0), (40, 100,0), (50, 100,0), (60, 100,0),
    (70, 100,0), (80, 100,0), (90, 100,0), (100, 100,0), (100, 90,0), (100, 80,0), (100, 70,0), (100, 60,0),
    (100, 50,0), (100, 40,0), (100, 30,0), (100, 20,0),  (100, 10,0), (100, 0,0),  (90, 0,0),   (80, 0,0),
    (70, 0,0),   (60, 0,0),   (50, 0,0),   (40, 0,0),    (30, 0,0),   (20, 0,0),   (10, 0,0),   (0, 0,0)
]

# Create Meshing Inputs
input_polygon = poly_input.PolyInput(outside_polygon=polygon_points)
mesh_io = MultiPolyMesherIo(polygons=[input_polygon])

# Generate 2dm
success, errors = mesh_utils.generate_2dm(mesh_io, 'output/out_file.2dm', 3)
print(success)
print(errors)

True



## check_mesh_input_topology()

The `check_mesh_input_topology()` function checks if the input polygons for a `MultiPolyMesherIo` object intersect one another. It takes a `MultiPolyMesherIo` object and returns a tuple containing a bool and a string of messages. The first entry in the tuple will hold `True` if the inputs are topologically correct and `False` otherwise. The second entry in the tuple will hold a string of messages when the topology is incorrect.

The example below shows `check_mesh_input_topology()` being used on a mesh that is topologically correct.

In [9]:
# Generate MultiPolyMesherIo
io = MultiPolyMesherIo(())
input_polygon = poly_input.PolyInput(outside_polygon=((0, 0, 0), (100, 0, 0), (100, 100, 0), (0, 100, 0)))
io.polygons = (input_polygon,)

# Check mesh topology
success, errors = mesh_utils.check_mesh_input_topology(io)
print(success)
print(errors)

True



The example below shows `check_mesh_input_topology()` being used on a mesh that IS NOT topologically correct (there is an intersection).

In [10]:
# Generate MultiPolyMesherIo
io = MultiPolyMesherIo(())
input_polygon = poly_input.PolyInput(outside_polygon=((0, 0, 0), (100, 0, 0), (100, 10, 0), (0, -10, 0)))
io.polygons = (input_polygon,)

# Check mesh topology
success, errors = mesh_utils.check_mesh_input_topology(io)
print(success)
print(errors)

False
Error: Input polygon segments intersect. The segment defined by points 0 and 1 of outer polygon 0 intersects with the segment defined by points 2 and 3 of outer polygon 0.



## redistribute_poly_line()

The `redistribute_poly_line()` function redistributes the points along a line to a constant spacing. It takes the two following arguments:

- `polyline` (iterable): Input poly line locations.
- `size` (float): The desired spacing for point redistribution.

The return value is a numpy array which holds the redistributed poly line locations.

In [11]:
# Create the locations for the input poly line
polygon_corners = [(0, 0, 0), (0, 100, 0), (100, 100, 0), (100, 0, 0), (0, 0, 0)]

# Redistribute the poly line
mesh_utils.redistribute_poly_line(polygon_corners, 25)

array([[  0.,   0.,   0.],
       [  0.,  25.,   0.],
       [  0.,  50.,   0.],
       [  0.,  75.,   0.],
       [  0., 100.,   0.],
       [ 25., 100.,   0.],
       [ 50., 100.,   0.],
       [ 75., 100.,   0.],
       [100., 100.,   0.],
       [100.,  75.,   0.],
       [100.,  50.,   0.],
       [100.,  25.,   0.],
       [100.,   0.,   0.],
       [ 75.,   0.,   0.],
       [ 50.,   0.,   0.],
       [ 25.,   0.,   0.],
       [  0.,   0.,   0.]])